 Exploring Keras library's role in computer vision. Load and convert images with Keras, how to use the MNIST handwritten image classification dataset and how to employ the ImageDataGenerator class.

In [12]:
# clone github repo
!git clone https://github.com/zaka-ai/computer-vision-course.git

fatal: destination path 'computer-vision-course' already exists and is not an empty directory.


 Load an Image with Keras

In [13]:
# example of loading an image with the Keras API
from tensorflow.keras.preprocessing.image import load_img

# load the image
img_path = "computer-vision-course/deep_learning/data/image1.jpg"
img = load_img(img_path)

# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)

<class 'PIL.JpegImagePlugin.JpegImageFile'>
JPEG
RGB
(640, 360)


 Convert an Image With Keras

In [14]:
# example of converting an image with the Keras API
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.preprocessing.image import array_to_img

# load the image
img_path = "computer-vision-course/deep_learning/data/image1.jpg"
img = load_img(img_path)
print(type(img))

# convert to numpy array
img_array = img_to_array(img) 
print(img_array.dtype)
print(img_array.shape)

# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
float32
(360, 640, 3)
<class 'PIL.JpegImagePlugin.JpegImageFile'>


ImageDataGenerator Class

In [15]:
# MNIST Handwritten Image Classification Dataset
from tensorflow.keras.datasets import mnist

(trainX, trainY), (testX, testY) = mnist.load_data()

# summarize dataset shape
print("Train", trainX.shape, trainY.shape)
print("Test", testX.shape, testY.shape)

# summarize pixel values
print("Train", trainX.min(), trainX.max(), trainX.mean(), trainX.std())
print("Test", testX.min(), testX.max(), testX.mean(), testX.std())

Train (60000, 28, 28) (60000,)
Test (10000, 28, 28) (10000,)
Train 0 255 33.318421449829934 78.56748998339798
Test 0 255 33.791224489795916 79.17246322228644


 Normalize Images With ImageDataGenerator

In [16]:
# reshape dataset to have a single channel because the ImageDataGenerator expects a numpy array of rank 4

# trainX: (60000, 28, 28) => (60000, 28, 28, 1)
# testX:  (10000, 28, 28) => (10000, 28, 28, 1)

trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2], 1))
testX = testX.reshape((testX.shape[0], trainX.shape[1], trainX.shape[2], 1))

In [17]:
print("Train", trainX.shape, trainY.shape)
print("Test", testX.shape, testY.shape)

Train (60000, 28, 28, 1) (60000,)
Test (10000, 28, 28, 1) (10000,)


In [18]:
# confirm scale of pixels
print(f'Train min= {trainX.min()}, max= {trainX.max()}')
print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))

Train min= 0, max= 255
Test min=0.000, max=255.000


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create generator (1.0/255.0)
datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators to scale images
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)

# 60000 / 64 = 937.5 ~= 938

print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

# confirm the scaling works
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000


 Center Images With ImageDataGenerator

In [20]:
# Find the mean of the pixel values and make sure all the pixels are distributed around it as if it were the new zero and the remaining values are positive and negative.

# report per-image mean
print('Means train=%.3f, test=%.3f' % (trainX.mean(), testX.mean()))

# create generator that centers pixel values
datagen = ImageDataGenerator(featurewise_center=True)

# calculate the mean on the training dataset
datagen.fit(trainX)
print('Data Generator Mean: %.3f' % datagen.mean)

# demonstrate effect on a single batch of samples
iterator = datagen.flow(trainX, trainY, batch_size=64)

# get a batch
batchX, batchy = iterator.next()

# mean pixel value in the batch
print(batchX.shape, batchX.mean())

Means train=33.318, test=33.791
Data Generator Mean: 33.318
(64, 28, 28, 1) -1.5882585


A better check would be to set the batch size to the size of the training dataset (e.g. 60,000 samples), retrieve one batch, then calculate the mean. It should be a very small value close to zero.

In [21]:
# demonstrate effect on entire training dataset
iterator = datagen.flow(trainX, trainY, batch_size=len(trainX), shuffle=False)

# get a batch
batchX, batchy = iterator.next()

# mean pixel value in the batch
print(batchX.shape, batchX.mean())

#display an image
img2 = batchX[41]
print(img2.shape)
from google.colab.patches import cv2_imshow
cv2_imshow(img2)

(60000, 28, 28, 1) -1.9512918e-05
(28, 28, 1)
